In [ ]:
from concurrent.futures import ThreadPoolExecutor
import pandas as pd
import requests
from bs4 import BeautifulSoup
import time
import json
import re

# Démarrage du chronomètre pour calculer la durée d'exécution du script
start_time = time.time()

# Définition de la variable pour pour suivre la progression
pourcent = 23

# Définition de la variable du nombre d'id dans l'url d'Allociné
total_film = 350000

# Définition de la fonction pour scraper les données d'Allociné
def allocine_scrap(i):
    global pourcent
    # Initialisation des variables pour les informations à récupérer
    acteurs = genre = realisateur = scenariste = duree = date = tite_original = note_presse = note_spectateurs = pays = distributeur = budget = None
    
    try:
        # Construction de l'URL pour chaque film basé sur son ID
        url = f'https://www.allocine.fr/film/fichefilm_gen_cfilm={i}.html'
        res = session.get(url, allow_redirects=True)

        # Print du % avec une intervalle de 24 tests
        pourcent += 1
        if pourcent == 24:
            print('Execution :', round((i/total_film)*100, 2), '%')
            pourcent = 0
        
        # Vérification du statut de la réponse, retourne des "0" si le code HTTP n'est pas 200 ou si l'URL a été redirigée
        if res.status_code != 200:
            return i, "0", "0", "0", "0", "0", "0", "0", "0", "0", "0", "0", "0", "0"
        if res.url != url:
            return i, "0", "0", "0", "0", "0", "0", "0", "0", "0", "0", "0", "0", "0"
        
        # Utilisation de BeautifulSoup pour parser le contenu HTML
        soup = BeautifulSoup(res.content, 'lxml')
        date = soup.find('span', class_=lambda x: x and 'date' in x)

        # Filtrage des films uniquement jusqu'au 31 décembre 2024
        if date and int(date.text.strip()[-4:]) < 2024:
            try:
                # Extraction et traitement du JSON contenant les infos du film
                script = soup.find('script', {'type': 'application/ld+json'})
                if script is None:
                    return i, "0", "0", "0", "0", "0", "0", "0", "0", "0", "0", "0", "0", "0"
                
                clean_json = script.string.replace('\n', '').replace('\r', '').replace('\t', '')
                df_json = json.loads(clean_json)

                # Extraction des acteurs
                acteurs = df_json.get("actor", [])
                if isinstance(acteurs, dict):
                    acteurs = [acteurs]
                acteurs = [i["name"] for i in acteurs]
                acteurs = ", ".join(acteurs)
                
                # Extraction des genres
                genre = df_json.get("genre")
                if not isinstance(genre, list):
                    genre = [genre]
                genre = ", ".join(genre)

                # Extraction du réalisateur
                realisateur = df_json.get("director", [])
                if isinstance(realisateur, dict):
                    realisateur = [realisateur]
                realisateur = [i["name"] for i in realisateur]
                realisateur = ", ".join(realisateur)
                
                # Extraction du scénariste
                scenariste = df_json.get("creator", [])
                if isinstance(scenariste, dict):
                    scenariste = [scenariste]
                scenariste = [i["name"] for i in scenariste]
                scenariste = ", ".join(scenariste)
                
                # Traitement de la durée du film
                duree = df_json.get("duration", None)
                if duree :
                    duree = duree.replace('PT', '').replace('M00S', '')
                    hours, minutes = duree.split('H')
                    duree = int(hours)*60+int(minutes)
                
            # Gestion des erreurs liées à JSON
            except json.JSONDecodeError as e:
                print(f"Erreur de décodage JSON pour l'indice {i}: {e}")
                print("JSON problématique:", script.string)
                return (i, "0", "0", "0", "0", "0", "0", "0", "0", "0", "0", "0", "0", "0")
            
            # Extraction du titre
            titre = soup.find('span', string='Titre original ')
            if titre:
                tite_original = titre.find_next_sibling(string=True).strip()

            # Extraction des notes
            notes = soup.find_all('span', class_='stareval-note')
            if notes:
                note_presse = notes[0].get_text(strip=True)
                note_spectateurs = notes[1].get_text(strip=True) if len(notes) > 1 else None
                
            # Extraction des informations techniques (Tableau en bas de page principale des films)
            info_tech = soup.find_all('span', class_='what light')
            pays, distributeur, budget = None, None, None
            for n in range(len(info_tech)):
                if info_tech[n].text.strip() == 'Nationalités' or info_tech[n].text.strip() == 'Nationalité':
                    span_1 = info_tech[n].find_next_sibling('span')
                    span_2 = span_1.find('span', class_=lambda x: x and 'nationality' in x)
                    if span_2:
                        pays = span_2.get_text(strip=True)
                if info_tech[n].text.strip() == 'Distributeur':
                    span_1 = info_tech[n].find_next_sibling('span', class_='that')
                    if span_1:
                        distributeur = span_1.get_text(strip=True)
                if info_tech[n].text.strip() == 'Budget':
                    span_1 = info_tech[n].find_next_sibling('span', class_='that')
                    if span_1:
                        budget = span_1.get_text(strip=True)
        # Gestion des film avec une date 2024 ou plus
        else:
            return i, "0", "0", "0", "0", "0", "0", "0", "0", "0", "0", "0", "0", "0"
            
    # Gestion des exceptions liées aux requêtes HTTP
    except requests.exceptions.RequestException:
        return i, "0", "0", "0", "0", "0", "0", "0", "0", "0", "0", "0", "0", "0"
    
    # Retourne les données extraites ou des "0" si les données ne sont pas disponibles
    resultat = (
        i,
        acteurs if acteurs else "0",
        genre if genre else "0",
        realisateur if realisateur else "0",
        scenariste if scenariste else "0",
        duree if duree else "0",
        date.text.strip() if date else "0",
        date.text.strip()[-4:] if date else "0",
        tite_original if tite_original else "0",
        note_presse if note_presse else "0",
        note_spectateurs if note_spectateurs else "0",
        pays if pays else "0",
        distributeur if distributeur else "0",
        budget if budget else "0",
    )
    print(f"Returning from allocine_scrap with {len(resultat)} values for index {i}")
    return resultat
# Initialisation d'un DataFrame pour stocker les données extraites   
df = pd.DataFrame(columns=['ID', 'acteurs', 'genre', 'realisateur', 'scenariste', 'duree', 'date', 'annee', 'titre_original', 'note_presse', 'note_spectateurs', 'pays', 'distributeur', 'budget']) 
session = requests.Session()

# Utilisation d'un ThreadPoolExecutor pour exécuter les scrapings en parallèle et utiliser les differents coeurs logique du PC.
with ThreadPoolExecutor(max_workers=8) as executor:
    # Soumission des tâches de scraping pour chaque ID de film entre 1 et la variable total_film
    tests = [executor.submit(allocine_scrap, i) for i in range(1, total_film)]
    for test in tests:
        # Récupération des résultats et ajout au DataFrame si le titre est différent de "0"
        i, acteurs, genre, realisateur, scenariste, duree, date, annee, tite_original, note_presse, note_spectateurs, pays, distributeur, budget = test.result()
        variables = (acteurs, genre, realisateur, scenariste, duree, date, annee, tite_original, note_presse, note_spectateurs, pays, distributeur, budget)
        if any(v != "0" for v in variables):
            df.loc[i] = [i, acteurs, genre, realisateur, scenariste, duree, date, annee, tite_original, note_presse, note_spectateurs, pays, distributeur, budget]

# Arrêt du chronomètre et affichage de la durée d'exécution
end_time = time.time()
execution_time = end_time - start_time
print(f"Le script a pris {execution_time} secondes pour s'exécuter.")

# Sauvegarde des résultats dans un fichier CSV
df.to_csv('allocine_PP.csv', index=False)

# Affichage du DataFrame final
display(df)